In [1]:
import sys

# REPO_PATH is the local git directory
REPO_PATH='/Users/Halmagyi/Documents/Programming/Python/ML/Kaggle/Planet/planet-kaggle'
sys.path.append(REPO_PATH)

import os
import numpy as np
import pandas as pd
from importlib import reload
import utils
from keras import backend as K

from skimage import io

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Here we import the py files for current project.
import utils
import models.vgglike as vl
import models.ekami as ek
import helpers.planet as helpers
import helpers.dfgen as gen
# from helpers.planet import PlanetData

reload(utils)
reload(vl)
reload(ek)
reload(helpers)
reload(gen)

<module 'helpers.dfgen' from '/Users/Halmagyi/Documents/Programming/Python/ML/Kaggle/Planet/planet-kaggle/helpers/dfgen.py'>

In [3]:
DATA_ROOT=os.environ.get('DATA')
DATA_DIR='planet'
WEIGHT_ROOT=os.environ.get('WEIGHTS')
IMG_TYPE='tif'
ROOT=f'{DATA_ROOT}/{DATA_DIR}'
WEIGHT_DIR=f'{WEIGHT_ROOT}/{DATA_DIR}' # Weights are saved and loaded here
JPG_DIR = os.path.join(ROOT, 'train-jpg')
TIF_DIR = os.path.join(ROOT, 'train-tif')

In [4]:
TAGS=[
    'primary',
    'clear',
    'agriculture',
    'road',
    'water',
    'partly_cloudy',
    'cultivation',
    'habitation',
    'haze',
    'cloudy',
    'bare_ground',
    'selective_logging',
    'artisinal_mine',
    'blooming',
    'slash_burn',
    'conventional_mine',
    'blow_down']

# Create  or import dataframe for sample

In [6]:
pld200=helpers.PlanetData(train_size=200,create=True)

In [5]:
pld200tag1=helpers.PlanetData(train_size=200,tags=['agriculture'],create=True)

TypeError: __init__() got an unexpected keyword argument 'tags'

In [ ]:
pld200tag1.train_df[:4]

In [ ]:
ls

In [29]:
pld200tag1.train_size,pld200tag1.valid_size

(200, 40)

In [30]:
train_gen=gen.DFGen(dataframe=pld200tag1.train_df,batch_size=32)

In [33]:
next(train_gen)

(array([[[[4397, 3668, 2348, 7900],
          [4298, 3563, 2282, 7829],
          [4439, 3671, 2431, 7681],
          ..., 
          [4366, 3531, 2331, 6629],
          [4297, 3457, 2264, 6620],
          [4123, 3364, 2160, 6719]],
 
         [[4476, 3688, 2389, 7869],
          [4448, 3647, 2435, 7664],
          [4569, 3725, 2500, 7422],
          ..., 
          [4312, 3506, 2273, 7083],
          [4223, 3462, 2249, 7008],
          [4210, 3462, 2234, 7016]],
 
         [[4391, 3636, 2365, 7912],
          [4316, 3541, 2278, 7610],
          [4432, 3701, 2465, 7266],
          ..., 
          [4244, 3460, 2206, 7308],
          [4202, 3472, 2222, 7216],
          [4231, 3536, 2261, 7192]],
 
         ..., 
         [[4261, 3570, 2303, 5962],
          [4180, 3487, 2236, 6197],
          [4113, 3415, 2189, 6351],
          ..., 
          [4074, 3254, 2096, 5895],
          [4056, 3279, 2100, 5948],
          [4111, 3322, 2126, 6239]],
 
         [[4274, 3545, 2342, 5239],
         

# Ekami Model: modified VGG

In [36]:
ek_bc_td1=ek.EKAMI(loss_func='binary_crossentropy',target_dim=1,metrics=['accuracy',utils.k_f2])

In [8]:
ek_bc.model().summary()

start with batch norm... (None, 256, 256, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 256, 256, 4)       16        
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 258, 258, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 32)      1184      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 130, 130, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (N

In [35]:
ek_bc_td1.fit_gen(epochs=1,pdata=pld200tag1)

start with batch norm... (None, 256, 256, 4)
Epoch 1/1
200/200 [==============================] - 984s - loss: 0.3824 - acc: 0.8366 - k_f2: 0.5943 - val_loss: 0.7694 - val_acc: 0.8459 - val_k_f2: 0.4922


In [ ]:
ek_bc_td1.predict_image()

# Primary Rainforest

In [37]:
pld200tag_ag=helpers.PlanetData(train_size=200,tags=['agriculture'],create=True)

In [39]:
ek_bc_td1.fit_gen(epochs=1,pdata=pld200tag_ag)

start with batch norm... (None, 256, 256, 4)
Epoch 1/1
200/200 [==============================] - 1267s - loss: 0.5214 - acc: 0.7406 - k_f2: 0.4238 - val_loss: 0.8139 - val_acc: 0.6599 - val_k_f2: 0.2911


In [44]:
ek_bc_td1.predict_image(name='train_17463',image_dir='train-tif',file_ext='tif')

array([[ 0.0496125]], dtype=float32)

In [47]:
ek_bc_td1.predict_image(name='train_23725',image_dir='train-tif',file_ext='tif')

array([[ 0.16808252]], dtype=float32)

In [48]:
pld200tag1.train_df['prediction']=pld200tag1.train_df.image_name.apply(lambda x: ek_bc_td1.predict_image(name=x,image_dir='train-tif',file_ext='tif'))

In [50]:
pld200tag1.train_df[:100]

,image_name,tags,vec,image_path,prediction
17463,train_17463,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.0496125]]
4223,train_4223,agriculture,[1],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.98978]]
25186,train_25186,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.0276857]]
23725,train_23725,agriculture,[1],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.168083]]
29646,train_29646,agriculture,[1],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.458181]]
39439,train_39439,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.0284381]]
39174,train_39174,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.911718]]
11490,train_11490,agriculture,[1],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.0754533]]
9613,train_9613,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.0372849]]
13701,train_13701,,[0],/Users/Halmagyi/Documents/Programming/Python/M...,[[0.388319]]
